In [1]:
import tensorflow as tf
import numpy as np
import pickle as pkl
import random
from Data_converter import convert_to_one_hot

Initialize parameters of the system
--------------------------

In [2]:
POOL_SIZE = 5
INPUT_CHANNEL = 220
OUTPUT_CLASSES = 16
KERNEL_1 = 21
FILTERS_1 = 20
HIDDEN_LAYER_1 = 100

Loading data
----------

In [3]:
f = pkl.load(open('our_data.pkl','rb'))
data = f['data']

Make next_batch function
---------------------

In [13]:
def batch(data,k):
	batch = random.sample(data, k)
	inputs =[]
	targets = []
	for a,b in batch:
		inputs.append(a)
		targets.append(b)
# 	targets = convert_to_one_hot(targets,OUTPUT_CLASSES)
	return inputs, targets


In [6]:
x = tf.placeholder("float",name='x',shape=([None,INPUT_CHANNEL,1,1]))
y = tf.placeholder("float",name='y',shape=([None,OUTPUT_CLASSES]))


Defining layers for our network
-----------------------

In [10]:
# """ define function """
# def weight_variable(shape):
# 	init = tf.truncated_normal(shape,stddev=1.0)
# 	return tf.Variable(init)

# def bias_variable(shape):
# 	init = tf.constant(0.1,shape=shape)
# 	return tf.Variable(init)

# def conv_2d(x,W):
# 	return tf.nn.conv2d(x,W,strides = [1,1,1,1], padding='VALID')

# def max_pool_5x1(x):
# 	return tf.nn.max_pool(x,ksize=[1,POOL_SIZE,1,1],strides=[1,POOL_SIZE,1,1],padding='SAME') # check k-size


Making the model
----------

__conv--ReLU--maxpool--fc__

In [11]:
def inference(input_data):
    
    with tf.variable_scope('h_conv1') as scope:
        weights = tf.get_variable('weights', shape=[KERNEL_1,1,1,FILTERS_1], 
                                  initializer=tf.contrib.layers.xavier_initializer_conv2d())
        biases = tf.get_variable('biases', shape=[FILTERS_1], initializer=tf.constant_initializer(0.05))
    
        z = tf.nn.conv2d(input_data, weights, strides=[1, 1, 1, 1], padding='VALID')
        h_conv1 = tf.nn.relu(z+biases, name=scope.name)
        
    h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, POOL_SIZE, 1, 1],
                        strides=[1, POOL_SIZE, 1, 1], padding='SAME', name='h_pool1')
    
    h_pool2_flat = tf.reshape(h_pool2, [-1, 40*1*20])
    
    with tf.variable_scope('h_fc') as scope:
        weights = tf.get_variable('weights', shape=[HIDDEN_LAYER_1, OUTPUT_CLASSES], 
                                  initializer=tf.contrib.layers.xavier_initializer())
        biases = tf.get_variable('biases', shape=[OUTPUT_CLASSES])
        
        logits = (tf.matmul(h_pool2_flat, weights) + biases)
    return logits

In [14]:
def calc_loss(logits, labels):
    """Calculates the loss from the logits and the labels.
    Args:
        logits: Logits tensor, float - [batch_size, NUM_CLASSES].
        labels: Labels tensor, int32 - [batch_size].
    Returns:
        loss: Loss tensor of type float.
    """
    labels = tf.to_float(labels)
    cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits, labels, name='xentropy')
    loss = tf.reduce_mean(cross_entropy, name='xentropy_mean')
    return loss

In [15]:
def training(loss, learning_rate=5e-4):
    """Sets up the training Ops.
    Creates a summarizer to track the loss over time in TensorBoard.
    Creates an optimizer and applies the gradients to all trainable variables.
    The Op returned by this function is what must be passed to the
    `sess.run()` call to cause the model to train.
    Args:
        loss: Loss tensor, from loss().
        learning_rate: The learning rate to use for gradient descent.
    Returns:
        train_op: The Op for training.
    """
    # Add a scalar summary for the snapshot loss.
    tf.scalar_summary(loss.op.name, loss)
    # Create the Adam optimizer with the given learning rate.
    optimizer = tf.train.AdamOptimizer(learning_rate)
    # Create a variable to track the global step.
    global_step = tf.Variable(0, name='global_step', trainable=False)
    # Use the optimizer to apply the gradients that minimize the loss
    # (and also increment the global step counter) as a single training step.
    train_op = optimizer.minimize(loss, global_step=global_step)
    return train_op

In [17]:
# def evaluation(logits, labels, topk=1):
#     """Evaluate the quality of the logits at predicting the label.
#     Args:
#         logits: Logits tensor, float - [batch_size, NUM_CLASSES].
#         labels: Labels tensor, int32 - [batch_size], with values in the
#                   range [0, NUM_CLASSES).
#         topk: the number k for 'top-k accuracy'
#     Returns:
#         A scalar int32 tensor with the number of examples (out of batch_size)
#         that were predicted correctly.
#     """
#     # For a classifier model, we can use the in_top_k Op.
#     # It returns a bool tensor with shape [batch_size] that is true for
#     # the examples where the label is in the top k (here k=1)
#     # of all logits for that example.
    
#     correct = tf.nn.in_top_k(logits, tf.reshape(tf.slice(labels, [0,1], [int(labels.get_shape()[0]), 1]),[-1]), topk)
#     # Return the number of true entries.
#     return tf.reduce_sum(tf.cast(correct, tf.int32))

In [18]:
def placeholder_inputs(batch_size):
    """Generate placeholder variables to represent the input tensors.
    
    These placeholders are used as inputs by the rest of the model building
    code and will be fed from the downloaded data in the .run() loop, below.
    Args:
        batch_size: The batch size will be baked into both placeholders.
    Returns:
        images_placeholder: Images placeholder.
        labels_placeholder: Labels placeholder.
    """
    # Note that the shapes of the placeholders match the shapes of the full
    # image and label tensors, except the first dimension is now batch_size
    # rather than the full size of the train or test data sets.
    images_placeholder = tf.placeholder(tf.float32, shape=([None,INPUT_CHANNEL,1,1]))
    labels_placeholder = tf.placeholder(tf.int32, shape=([None,OUTPUT_CLASSES]))
    return images_placeholder, labels_placeholder

In [19]:

#UPDATE current_img_ind
def fill_feed_dict(data_set, images_pl, labels_pl, current_img_ind, batch_size):
    """Fills the feed_dict for training the given step.
    A feed_dict takes the form of:
    feed_dict = {
                  <placeholder>: <tensor of values to be passed for placeholder>,
                  ....
                }
    Args:
        data_set: The set of images and labels, from input_data.read_data_sets()
        images_pl: The images placeholder, from placeholder_inputs().
        labels_pl: The labels placeholder, from placeholder_inputs().
        current_img_ind: The current position of the index in the dataset
    Returns:
        feed_dict: The feed dictionary mapping from placeholders to values.
        current_img_ind: The updated position of the index in the dataset
        data_set: updated data_set
    """
    # Create the feed_dict for the placeholders filled with the next
    # `batch size ` examples.
    batch, current_img_ind, data_set= next_batch(batch_size, data_set, current_img_ind)

    feed_dict = {
      images_pl: batch[0],
      labels_pl: batch[1]
    }
    return feed_dict, current_img_ind, data_set

In [20]:
def do_eval(sess, eval_correct, images_placeholder, labels_placeholder, data_set, batch_size):
    """Runs one evaluation against the full epoch of data.
    Args:
        sess: The session in which the model has been trained.
        eval_correct: The Tensor that returns the number of correct predictions.
        images_placeholder: The images placeholder.
        labels_placeholder: The labels placeholder.
        data_set: The set of images and labels to evaluate, from
                input_data.read_data_sets().
    """
    # And run one epoch of eval.
    true_count = 0  # Counts the number of correct predictions.
    steps_per_epoch = len(data_set) // batch_size
    num_examples = steps_per_epoch * batch_size
    current_img_ind = 0
    for step in xrange(steps_per_epoch):
        feed_dict, current_img_ind, data_set = fill_feed_dict(data_set, images_placeholder,
                               labels_placeholder, current_img_ind, batch_size)
        true_count += sess.run(eval_correct, feed_dict=feed_dict)
    precision = true_count / num_examples
    print('  Num examples: %d  Num correct: %d  Precision @ 1: %0.04f' %
            (num_examples, true_count, precision))

In [2]:
def run_training():
    """Train for a number of steps."""
    # Tell TensorFlow that the model will be built into the default Graph.
    train_data = data[:int(TRAINING_PROP*len(data))]
    test_data = data[int(TRAINING_PROP*len(data)):]
    train_data = train_data.reset_index(drop = True)
    test_data = test_data.reset_index(drop = True)
    
    with tf.Graph().as_default():
        # Generate placeholders for the images and labels.
        images_placeholder, labels_placeholder = placeholder_inputs(BATCH_SIZE)

        # Build a Graph that computes predictions from the inference model.
        logits = inference(images_placeholder, 0.5, 512)

        # Add to the Graph the Ops for loss calculation.
        loss = calc_loss(logits, labels_placeholder)

        # Add to the Graph the Ops that calculate and apply gradients.
        train_op = training(loss, LEARNING_RATE)

        # Add the Op to compare the logits to the labels during evaluation.
        eval_correct = evaluation(logits, labels_placeholder)

        # Build the summary operation based on the TF collection of Summaries.
        #summary_op = tf.merge_all_summaries()

        # Create a saver for writing training checkpoints.
        saver = tf.train.Saver()

        # Create a session for running Ops on the Graph.
        sess = tf.Session()

        # Run the Op to initialize the variables.
        init = tf.initialize_all_variables()
        sess.run(init)

        # Instantiate a SummaryWriter to output summaries and the Graph.
        #summary_writer = tf.train.SummaryWriter('../Data/', sess.graph)
        current_img_ind = 0
        # And then after everything is built, start the training loop.
        for step in xrange(MAX_STEPS):
            start_time = time.time()

            # Fill a feed dictionary with the actual set of images and labels
            # for this particular training step.
            feed_dict, current_img_ind, train_data = fill_feed_dict(train_data,
                                 images_placeholder,
                                 labels_placeholder, current_img_ind=current_img_ind, 
                                                        batch_size=BATCH_SIZE)
            

            # Run one step of the model.  The return values are the activations
            # from the `train_op` (which is discarded) and the `loss` Op.  To
            # inspect the values of your Ops or variables, you may include them
            # in the list passed to sess.run() and the value tensors will be
            # returned in the tuple from the call.
            _, loss_value = sess.run([train_op, loss],
                               feed_dict=feed_dict)

            duration = time.time() - start_time

            # Write the summaries and print an overview fairly often.
            if step % 5 == 0:
                # Print status to stdout.
                print('Step %d: loss = %.2f (%.3f sec)' % (step, loss_value, duration))
                # Update the events file.
                #summary_str = sess.run(summary_op, feed_dict=feed_dict)
                #summary_writer.add_summary(summary_str, step)
                #summary_writer.flush()

            # Save a checkpoint and evaluate the model periodically.
            if (step + 1) % (50) == 0 or (step + 1) == MAX_STEPS:
                saver.save(sess, '../Data/model.ckpt')
                # Evaluate against the training set.
                print('Training Data Eval:')
                do_eval(sess, eval_correct, images_placeholder, labels_placeholder, train_data, BATCH_SIZE)
                # Evaluate against the validation set.
                print('Validation Data Eval:')
                do_eval(sess, eval_correct, images_placeholder, labels_placeholder, test_data, BATCH_SIZE)